In [20]:
import re

import tensorflow as tf
from tensorflow import keras
from keras import Model, Sequential
from keras.layers import (Input, Dense, Embedding, Flatten, Concatenate,
                          Dropout, Normalization, StringLookup)
from keras.optimizers.legacy import Adam # Metal only supports legacy Adam
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

from utils import DataSet

##### Option 3: Use Embedding of selfies structure

We will create an embedding of the "SELFIE" tokens using  `tensorflow` [StringLookup](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup) to encode the selfie tokens into consistent categorical int values, and [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) to create a the embedding vector for each Chromophore.

The `y` dataframe already includes the encoded `selfie` string for each chromophore, but we will need to pad the tokens to a max-length as the Emdedding requires that all the feature vectors be of the same length.

We could use the `selfies` module inbuild functions to split the selfies and create a standard vocab or alphabet of tokens that we will see, but they are a little clunky as they return consumable generator objects.  So using simple regular expression to do the same job.

We can add extra 'buckets' for 'out-of-vocab' tokens that could be present in molecules not in the training set, and we need to add a special 'mask token' `[nop]` to represent a padded token that should be ignored.

In [26]:
# Helper functions to tokenize selfies
def split_selfie(selfie):
    """Split a selfie string into a list of tokens."""
    tokens = re.findall(r"(\[.*?\])", selfie)
    return tokens

def pad_selfie(selfie, pad_length):
    """Pad a selfie to a standard length and convert to a tensor."""
    tokens = split_selfie(selfie)
    padding = ["[nop]"] * (pad_length - len(tokens))
    tokens = tokens + padding
    return tf.convert_to_tensor(tokens, dtype=tf.string)


def pad_selfies_tokens(selfies, pad_length):
    """Pad a list of tokens to a standard length and convert to a tensor."""
    selfies_tokens = [pad_selfie(selfie, pad_length) for selfie in selfies]
    return tf.convert_to_tensor(selfies_tokens, dtype=tf.string)

In [27]:
# Load the dataset for this specific notebook
def notebook_dataset():
    lec_data = DataSet(
        target="LogExtCoeff",
        fill_na="drop",
        drop_na_selfies=True,
        descriptors="continuous",
        drop_features=['Ipc']
    )
    return lec_data

lec_data = notebook_dataset()

In [4]:
# Tokenize the SELFIES
selfie_tokens = [split_selfie(s) for s in lec_data.y["SELFIES"]]

# Calculate the maximum length of the SELFIES
max_len = max([len(tokens) for tokens in selfie_tokens])
print(f"Max length: {max_len}")


# Create the train and test data
y_train = lec_data.y_train["LogExtCoeff"]
y_test = lec_data.y_test["LogExtCoeff"]
sf_train = pad_selfies_tokens(lec_data.y_train["SELFIES"], max_len)
sf_test = pad_selfies_tokens(lec_data.y_test["SELFIES"], max_len)
x_train = lec_data.X_train
x_test = lec_data.X_test

Max length: 283


2023-07-18 10:52:20.877706: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-07-18 10:52:20.877730: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-07-18 10:52:20.877734: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-07-18 10:52:20.877771: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-18 10:52:20.877787: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


The limit of max-length will potentially cause issues if encountering longer molecules in any explored chemical space.  Could set larger - but not certain this will help accurancy as the model would never have been trained on any samples of the extended length.  Unclear to me how a RNN would change this ...

In [28]:
#### SELFIE MODEL ####
# Create a vocabulary of all the SELFIES tokens
vocab = list(set([token for tokens in selfie_tokens for token in tokens]))
print(f"Number of token_vocab: {len(vocab)}")

# Create a single-input model with the embedded SELFIES
num_oov = 2
lookup_params = {
    "vocabulary": vocab,
    "mask_token": "[nop]",
    "num_oov_indices": num_oov,
    "name": "SELFIES_LOOKUP",
}
embed_params = {
    "input_dim": len(vocab) + num_oov,
    "output_dim": 32,
    "mask_zero": True,
    "name": "SELFIES_EMBEDDING",
}

Number of token_vocab: 68


In [6]:
selfie_model = Sequential(
    [
        Input(shape=(max_len,), dtype=object, name="SELFIES"),
        StringLookup(**lookup_params),
        Embedding(**embed_params),
        Flatten(),
        Dense(32, activation='relu', name="HIDDEN_0"),
        # Dropout(0.5),
        # Dense(4, activation='relu', name="HIDDEN_1"),
        Dropout(0.1),
        Dense(1, name="OUTPUT"),
    ]
)

# Compile the model
selfie_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
)
selfie_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SELFIES_LOOKUP (StringLook  (None, 283)               0         
 up)                                                             
                                                                 
 SELFIES_EMBEDDING (Embeddi  (None, 283, 32)           2240      
 ng)                                                             
                                                                 
 flatten (Flatten)           (None, 9056)              0         
                                                                 
 HIDDEN_0 (Dense)            (None, 32)                289824    
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 OUTPUT (Dense)              (None, 1)                 3

In [35]:
# Train the models
hist_selfie = selfie_model.fit(
    sf_train,
    y_train,
    validation_split=0.2,
    epochs=25,
    batch_size=64,
)
selfie_model_pred = selfie_model.predict(sf_test)
print(f"\n\nR2 score selfie only: {r2_score(y_test, selfie_model_pred)}")
print(f"MSE score selfie only: {mean_squared_error(y_test, selfie_model_pred)}\n\n")

Epoch 1/25


2023-07-17 19:35:03.064988: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 10s 124ms/step - loss: 1.2408 - val_loss: 0.2713
Epoch 2/25


2023-07-17 19:35:13.174248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 10s 121ms/step - loss: 0.3304 - val_loss: 0.2171
Epoch 3/25
81/81 [==============================] - 10s 118ms/step - loss: 0.2869 - val_loss: 0.1941
Epoch 4/25
81/81 [==============================] - 10s 119ms/step - loss: 0.2512 - val_loss: 0.1673
Epoch 5/25
81/81 [==============================] - 10s 120ms/step - loss: 0.2261 - val_loss: 0.1588
Epoch 6/25
81/81 [==============================] - 10s 119ms/step - loss: 0.2124 - val_loss: 0.1529
Epoch 7/25
81/81 [==============================] - 10s 119ms/step - loss: 0.1955 - val_loss: 0.1494
Epoch 8/25
81/81 [==============================] - 10s 120ms/step - loss: 0.1844 - val_loss: 0.1673
Epoch 9/25
81/81 [==============================] - 10s 120ms/step - loss: 0.1747 - val_loss: 0.1441
Epoch 10/25
81/81 [==============================] - 10s 120ms/step - loss: 0.1673 - val_loss: 0.1482
Epoch 11/25
81/81 [==============================] - 10s 119ms/step - loss: 0.1567 - val_loss: 0.1439

2023-07-17 19:39:03.883881: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [8]:
#### DESCRIPTOR INPUT MODEL ####

# Base line of the descriptor model
scaler = StandardScaler().fit(x_train)
estimator = Ridge(alpha=0.055, random_state=42)
estimator.fit(scaler.transform(x_train), y_train)
est_pred = estimator.predict(scaler.transform(x_test))
print(f"\n\nR2 score single: {r2_score(y_test, est_pred):.2f}")
print(f"MSE score single: {mean_squared_error(y_test, est_pred):.2f}\n\n")



R2 score single: 0.42
MSE score single: 0.21




In [35]:
# Reset the data
lec_data = notebook_dataset()

# Normalization layer
normal_layer = Normalization()
normal_layer.adapt(x_train)

# Model
desc_model = Sequential(
    [
        Input(shape=(x_train.shape[1],), name="DESCRIPTORS"),
        normal_layer,
        # tf.keras.layers.Dense(32, activation='relu', name="HIDDEN_0"),
        # tf.keras.layers.Dropout(0.3),
        # tf.keras.layers.Dense(16, activation='relu', name="HIDDEN_1"),
        # tf.keras.layers.Dropout(0.3),
        # tf.keras.layers.Dense(8, activation='relu', name="HIDDEN_2"),
        # tf.keras.layers.Dropout(0.2),
        Dense(1, name="OUTPUT"),
    ]
)

# Compile the model
desc_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='mean_squared_error',
)

desc_model.summary()

2023-07-18 11:06:28.721042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-18 11:06:28.735663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_10 (Normaliz  (None, 104)               209       
 ation)                                                          
                                                                 
 OUTPUT (Dense)              (None, 1)                 105       
                                                                 
Total params: 314 (1.23 KB)
Trainable params: 105 (420.00 Byte)
Non-trainable params: 209 (840.00 Byte)
_________________________________________________________________


In [36]:
# Train the models
hist_desc = desc_model.fit(
    x_train,
    y_train,
    validation_split=0.20,
    epochs=50,
    batch_size=64,
)

desc_model_pred = desc_model.predict(x_test)
print(f"\n\nR2 score descriptors only: {r2_score(y_test, desc_model_pred)}")
print(f"MSE score descriptors only: {mean_squared_error(y_test, desc_model_pred)}")

Epoch 1/50
19/81 [======>.......................] - ETA: 0s - loss: 20.2634

2023-07-18 11:06:32.770263: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 1s 7ms/step - loss: 17.0604 - val_loss: 13.5259
Epoch 2/50
12/81 [===>..........................] - ETA: 0s - loss: 13.4781

2023-07-18 11:06:33.327675: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 0s 6ms/step - loss: 11.2733 - val_loss: 8.8724
Epoch 3/50
81/81 [==============================] - 0s 6ms/step - loss: 7.1895 - val_loss: 5.6917
Epoch 4/50
81/81 [==============================] - 0s 5ms/step - loss: 4.4280 - val_loss: 3.3730
Epoch 5/50
81/81 [==============================] - 0s 5ms/step - loss: 2.6555 - val_loss: 2.0459
Epoch 6/50
81/81 [==============================] - 0s 5ms/step - loss: 1.5719 - val_loss: 1.2285
Epoch 7/50
81/81 [==============================] - 0s 5ms/step - loss: 0.8972 - val_loss: 0.6779
Epoch 8/50
81/81 [==============================] - 0s 6ms/step - loss: 0.5462 - val_loss: 0.4335
Epoch 9/50
81/81 [==============================] - 0s 5ms/step - loss: 0.3710 - val_loss: 0.3215
Epoch 10/50
81/81 [==============================] - 0s 5ms/step - loss: 0.2856 - val_loss: 0.2540
Epoch 11/50
81/81 [==============================] - 0s 5ms/step - loss: 0.2428 - val_loss: 0.2356
Epoch 12/50
81/81 [=========

2023-07-18 11:06:54.534466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:

# Save the models
desc_model.save('models/lec_desc_only_model.keras')

In [ ]:
### MULTI INPUT MODEL ####

# Reset the data
lec_data = notebook_dataset()

# Now create a multi-input model with the embedded SELFIES and the other features using Functional API
selfie_input_layer = Input(shape=(max_len,), dtype=object, name="SELFIES")
lookup_layer = StringLookup(**lookup_params)(selfie_input_layer)
embed_layer = Embedding(**embed_params)(lookup_layer)
embed_flat_layer = Flatten(name="SELFIES_FLAT")(embed_layer)


# Create a new input layer for the continuous features
descriptors_layer = Input(shape=(x_train.shape[1],), name="DESCRIPTORS")

# Create a normalization layer for the continuous features
normalized_layer = normal_layer(descriptors_layer)

# Concatenate the embedded SELFIES and the continuous features
concat_layer = Concatenate()([embed_flat_layer, normalized_layer])

# Add the regression layers
hidden_layer = Dense(256, activation='relu', name="HIDDEN_0")(concat_layer)
dropout_layer = Dropout(0.4)(hidden_layer)
output_layer = Dense(1, name="OUTPUT")(dropout_layer)

# Create the model
multi_input_model = Model(inputs=[selfie_input_layer, descriptors_layer], outputs=output_layer)

# Compile the model
multi_input_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
)
multi_input_model.summary()

In [ ]:
# Train the model
hist_multi = multi_input_model.fit(
    {"SELFIES": sf_train, "DESCRIPTORS": x_train},
    y_train,
    validation_split=0.20,
    epochs=10,
    batch_size=64
)
multi_input_model_pred = multi_input_model.predict([sf_test, x_test])
print(f"R2 score multi: {r2_score(y_test, multi_input_model_pred)}")
print(f"RMSE score multi: {mean_squared_error(y_test, multi_input_model_pred)}")

# Save the model
multi_input_model.save('models/lec_selfies_multi_model.keras')





